# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load New

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [2]:
import pandas as pd
import config as cfg
from tqdm import tqdm
import swifter
from pshmodule.utils import filemanager as fm

In [3]:
df1 = fm.load(cfg.data_path_v1)
df2 = fm.load(cfg.data_path_v2)

extension : .xlsx
Loaded 21962 records from drive/MyDrive/MemeProject/data/row_data/meme_v1.xlsx
extension : .xlsx
Loaded 14640 records from drive/MyDrive/MemeProject/data/row_data/meme_v2.xlsx


In [4]:
new_header = df1.iloc[1]
df1 = df1[2:]
df1.columns = new_header

Merge - 관리 번호 df1 A, E 제외, df2 A, E

In [5]:
df1 = df1[['관리번호', 'utt',	'발화구분',	'발화문',	'ARG1',	'ARG_op']]
df2 = df2[['관리번호', 'utt',	'발화구분',	'발화문',	'ARG1',	'ARG_op']]

In [6]:
print(len(df1))
print(len(df2))

21960
14640


In [7]:
df_A = df2[df2['관리번호'].str.startswith('A')]
df_E = df2[df2['관리번호'].str.startswith('E')]

In [8]:
df = pd.concat([df_A, df1, df_E])
df.reset_index(inplace=True, drop=True)

Numbering

In [9]:
# no
num_list = [i+1 for i in range(len(df))]
df['num'] = num_list

# group no
g_list = []
num = 1
for i in range(len(df)):
  if i % 24 == 0 and i != 0:
    num += 1
  g_list.append(num)

df['g_num'] = g_list

In [10]:
df = df[['num', 'g_num', 'utt', '발화구분', '발화문', 'ARG1', 'ARG_op']]
df.rename(columns={'발화구분':'type', '발화문':'u', 'ARG1':'arg1', 'ARG_op':'arg_op'}, inplace=True)
df = df.fillna('')

<ipython-input-10-79048a69353d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'발화구분':'type', '발화문':'u', 'ARG1':'arg1', 'ARG_op':'arg_op'}, inplace=True)


In [12]:
df.tail()

,num,g_num,utt,type,u,arg1,arg_op
36595,36596,1525,20,CPO,,,
36596,36597,1525,21,CPO,,,
36597,36598,1525,22,CPO,,,
36598,36599,1525,23,CPO,,,
36599,36600,1525,24,CPO,,,


Reshape

In [13]:
result = []
temp = []

for i in tqdm(df.iterrows()):
  temp.append([i[1]['num'], i[1]['g_num'], i[1]['utt'], i[1]['type'], i[1]['u'], i[1]['arg1'], i[1]['arg_op']])
  
  if i[1]['num'] % 24 == 0 and i[1]['num'] != 0:
    temp_u = temp[:6]
    temp_cpr = temp[6:12]
    temp_mm = temp[12:18]
    temp_cpo = temp[18:24]

    for u in temp_u:
      # 밈 조합
      for cpr in temp_cpr:
        for mm in temp_mm:
          for cpo in temp_cpo:
            meme = "{} {} {}".format(cpr[4], mm[4], cpo[4]).strip()
            result.append([u[0], u[1], u[2], u[4], meme, u[5], u[6]])
    # 24개 씩 초기화
    temp = []

36600it [00:09, 3661.39it/s]


In [14]:
len(result)

1976400

In [89]:
df = pd.DataFrame(result, columns=[['num', 'g_num', 'u_num', 'u', 'meme', 'arg1', 'arg_op']])

In [90]:
df.head()

,num,g_num,u_num,u,meme,arg1,arg_op
0,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
1,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
2,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
3,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,
4,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,


arg_op 쉼표 기준으로 쪼개기

In [91]:
df['arg_op1'] = df.arg_op.swifter.apply(lambda x: x.str.split(',').str[0])
df['arg_op2'] = df.arg_op.swifter.apply(lambda x: x.str.split(',').str[1])

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [99]:
df.fillna('', inplace=True)

In [92]:
df.drop_duplicates(inplace=True)

In [93]:
df.reset_index(inplace=True, drop=True)

In [100]:
df.tail()

,num,g_num,u_num,u,meme,arg1,arg_op,arg_op1,arg_op2
181699,36582,1525,6,전국노래자랑에서 이모가 1등했다,대길이 이모님... 힘숨찐ㄷㄷ 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서,전국노래자랑에서,
181700,36582,1525,6,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시네! 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서,전국노래자랑에서,
181701,36582,1525,6,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시라능 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서,전국노래자랑에서,
181702,36582,1525,6,전국노래자랑에서 이모가 1등했다,대길좌 이모님이 우승자라구?! 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서,전국노래자랑에서,
181703,36582,1525,6,전국노래자랑에서 이모가 1등했다,"대길쓰 이모님,, 힘숨찐ㄷㄷ 상상도 못한 정체 ㄴㅇㄱ",,전국노래자랑에서,전국노래자랑에서,


Temp Save

In [104]:
fm.save(cfg.aug_temp, df)

Saved 181704 records


# Replace & Augmentation

In [105]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [106]:
from tqdm import tqdm
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [107]:
df = fm.load(cfg.aug_temp)

extension : .pickle
Loaded 181704 records from drive/MyDrive/MemeProject/data/augmentation/aug_temp.pickle


In [ ]:
df.tail(50)

,num,u,answer,arg1,arg_op,num2
208650,5796,비상계단으로 15층까지 걸어갈거야,대길쓰.. 후회할걸..? 스불재..!,비상계단,15층까지,34777
208651,5796,비상계단으로 15층까지 걸어갈거야,대길쓰.. 후회할걸..? 이것은 스불재..,비상계단,15층까지,34777
208652,5796,비상계단으로 15층까지 걸어갈거야,대길쓰 분명 후회한다ㄷㄷ 스불재..,비상계단,15층까지,34777
208653,5796,비상계단으로 15층까지 걸어갈거야,대길쓰 분명 후회한다ㄷㄷ 스불재 예상함니다,비상계단,15층까지,34777
208654,5796,비상계단으로 15층까지 걸어갈거야,대길쓰 분명 후회한다ㄷㄷ 스불재,비상계단,15층까지,34777
208655,5796,비상계단으로 15층까지 걸어갈거야,대길쓰 분명 후회한다ㄷㄷ 스불재 예상이요~,비상계단,15층까지,34777
208656,5797,비상계단으로 15층까지 걸어갈거야,대길쓰 분명 후회한다ㄷㄷ 스불재..!,비상계단,15층까지,34778
208657,5797,비상계단으로 15층까지 걸어갈거야,대길쓰 분명 후회한다ㄷㄷ 이것은 스불재..,비상계단,15층까지,34778
208658,5797,눈썹헤나 엄마한테 시술받아야지,오~ 탈룰라 조심해야겠어 껄껄,눈썹헤나,엄마한테,34778
208659,5797,눈썹헤나 엄마한테 시술받아야지,오... 탈룰라 조심해야겠다 껄껄,눈썹헤나,엄마한테,34778


##### numbering

##### 216 마다 처리


In [ ]:
num = []
u = []
answer = []
arg1 = []
arg_op = []
result = []


for i in tqdm(df.iterrows()):
  num.append(i[1]['num'])
  u.append(i[1]['u'])
  answer.append(i[1]['answer'])
  arg1.append(i[1]['arg1'])
  arg_op.append(i[1]['arg_op'])

  if i[0] % 215 == 0 and i[0] != 0:
    # 문장 6개씩 조합
    set_arg1 = set(arg1)
    set_argop = set(arg_op)
    set_arg1.discard('')
    set_argop.discard('')

    for z in zip(num, u, answer):
      # arg1 교체
      arg1_split = z[1].split()
      for k, s in enumerate(arg1_split):
        if not set_arg1:
          result.append([z])
        elif s in set_arg1:
          for w in set_arg1:
            arg1_split[k] = w
            z1 = ' '.join(arg1_split)

            # arg_op 교체
            argop_split = z1.split()
            for kk, ss in enumerate(argop_split):
              if not set_argop:
                result.append([z[0], z1, z[2], '', ''])
              elif ss in set_argop:
                for ww in set_argop:
                  argop_split[kk] = ww
                  z2 = ' '.join(argop_split)
                  result.append([z[0], z2, z[2], w, ww])

    # init
    num.clear()
    u.clear()
    answer.clear()
    arg1.clear()
    arg_op.clear()

In [ ]:
print(df.shape)
print(len(result))

save reference data set

In [ ]:
df_result = pd.DataFrame(result, columns=[['s_num', 'utterance', 'meme', 'arg1', 'arg_op']])

In [ ]:
df_result.head(20)

In [ ]:
fm.save(cfg.ref_qa_path, df_result)

# Utterance Save

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [ ]:
import config as cfg
from pshmodule.utils import filemanager as fm

원본

In [ ]:
df = fm.load(cfg.aug_temp)

extension : .pickle
Loaded 208700 records from drive/MyDrive/MemeProject/data/augmentation/aug_temp.pickle


In [ ]:
df = df[['u']]
df = df.drop_duplicates()

In [ ]:
df.shape

(10317, 1)

In [ ]:
df.tail()

,num,u
208676,5797,전국노래자랑에서 우리 이모가 우승탔다구
208682,5797,전국노래자랑에서 막내이모가 대상탔음
208688,5797,울 이모가 전국노래자랑에서 대상받았자나
208692,5798,울 이모가 전국노래자랑에서 대상받았자나
208694,5798,전국노래자랑에서 이모가 1등했다


In [ ]:
df_u = df.u.drop_duplicates()

In [ ]:
df_u.shape

(10317,)

In [ ]:
df.tail()

,num,u,answer,arg1,arg_op
208695,5798,전국노래자랑에서 이모가 1등했다,대길이 이모님... 힘숨찐ㄷㄷ 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208696,5798,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시네! 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208697,5798,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시라능 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208698,5798,전국노래자랑에서 이모가 1등했다,대길좌 이모님이 우승자라구?! 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208699,5798,전국노래자랑에서 이모가 1등했다,"대길쓰 이모님,, 힘숨찐ㄷㄷ 상상도 못한 정체 ㄴㅇㄱ",,전국노래자랑에서


In [ ]:
df_result = fm.load(cfg.ref_qa_path)

extension : .pickle
Loaded 1282547 records from drive/MyDrive/MemeProject/data/augmentation/meme_ref_qa.pickle


In [ ]:
df_result.tail(30)

,s_num,utterance,meme,arg1,arg_op
1282517,5794,생각보다 전체 회식 사용할거야,후회각일듯ㄷㄷ 스불재..,전체 회식,생각보다
1282518,5794,15층 우리집까지 전체 회식 사용할거야,후회각일듯ㄷㄷ 스불재..,전체 회식,15층 우리집까지
1282519,5794,관광스팟에서 전체 회식 사용할거야,후회각일듯ㄷㄷ 스불재..,전체 회식,관광스팟에서
1282520,5794,한겨울에도 전체 회식 사용할거야,후회각일듯ㄷㄷ 스불재..,전체 회식,한겨울에도
1282521,5794,겨울 취향 사용할거야,후회각일듯ㄷㄷ 스불재..,취향,겨울
1282522,5794,좀 취향 사용할거야,후회각일듯ㄷㄷ 스불재..,취향,좀
1282523,5794,15층까지 취향 사용할거야,후회각일듯ㄷㄷ 스불재..,취향,15층까지
1282524,5794,한겨울 취향 사용할거야,후회각일듯ㄷㄷ 스불재..,취향,한겨울
1282525,5794,겨울에도 취향 사용할거야,후회각일듯ㄷㄷ 스불재..,취향,겨울에도
1282526,5794,관광장소에서 취향 사용할거야,후회각일듯ㄷㄷ 스불재..,취향,관광장소에서


In [ ]:
df_result = df_result[['s_num', 'utterance']]
df_result.utterance.drop_duplicates(inplace=True)

<ipython-input-41-0bb0e6159faf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result.utterance.drop_duplicates(inplace=True)


In [ ]:
df_result.head(20)

,s_num,utterance
0,1,나 중간 반에서 1등했어
1,1,나 중간 반 1등했어
2,1,나 중간 우리 반에서 1등했어
3,1,나 시험 반에서 1등했어
4,1,나 시험 반 1등했어
5,1,나 시험 우리 반에서 1등했어
6,1,나 중간고사 반에서 1등했어
7,1,나 중간고사 반 1등했어
8,1,나 중간고사 우리 반에서 1등했어
9,1,나 중간시험 반에서 1등했어
